In [1]:
import numpy as np

In [2]:
xy_train_validate = np.load('./splitted_data/xy_train_validate.npy')
xy_test = np.load('./splitted_data/xy_test.npy')

In [7]:
file1 = open("xy_train_validate.txt", "w")
for line in xy_train_validate:
    file1.write(line +'\n')
file1.close()

In [6]:
list(xy_train_validate)

['A very clean and well decorated empty bathroom',
 'A panoramic view of a kitchen and all of its appliances.',
 'A blue and white bathroom with butterfly themed wall tiles.',
 'A panoramic photo of a kitchen and dining room',
 'A graffiti-ed stop sign across the street from a red car ',
 'A vandalized stop sign and a red beetle on the road',
 'A bathroom with a border of butterflies and blue paint on the walls above it.',
 'An angled view of a beautifully decorated bathroom.',
 'The two people are walking down the beach.',
 'A sink and a toilet inside a small bathroom.',
 'An empty kitchen with white and black appliances.',
 'A white square kitchen with tile floor that needs repairs ',
 'The vanity contains two sinks with a towel for each.',
 'Several metal balls sit in the sand near a group of people.',
 'Two people carrying surf boards on a beach.',
 'A kitchen with brown cabinets, tile backsplash, and grey counters.',
 'A surfer, a woman, and a child walk on the beach.',
 'A few pe

In [8]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

In [9]:
!pip install tensorflow==1.14
!pip install fastBPE
!pip install transformers
!pip install gsutil

In [ ]:
!wget -O moby_dick.txt https://www.gutenberg.org/files/2701/2701-0.txt

In [11]:
!wget -O vocab https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
!mv vocab ../
!wget -O codes https://raw.githubusercontent.com/salesforce/ctrl/master/codes
!mv codes ../
!wget -O generation.py https://raw.githubusercontent.com/salesforce/ctrl/master/generation.py
!mv generation.py ../
!wget -O generation.py https://raw.githubusercontent.com/salesforce/ctrl/master/generation.py
!wget -O control_codes.py https://raw.githubusercontent.com/salesforce/ctrl/master/control_codes.py
!wget -O vocab https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
!wget -O codes https://raw.githubusercontent.com/salesforce/ctrl/master/codes
!wget -O convert_tf_to_huggingface_pytorch.py https://raw.githubusercontent.com/salesforce/ctrl/master/convert_tf_to_huggingface_pytorch.py
!wget -O make_tf_records.py https://raw.githubusercontent.com/salesforce/ctrl/master/training_utils/make_tf_records.py
!wget -O training.py https://raw.githubusercontent.com/salesforce/ctrl/master/training_utils/training.py
!wget -O transformer.py https://raw.githubusercontent.com/salesforce/ctrl/master/transformer.py

--2021-01-23 21:47:30--  https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2996416 (2.9M) [text/plain]
Saving to: ‘vocab’

vocab               100%[===================>]   2.86M  --.-KB/s    in 0.07s   

2021-01-23 21:47:31 (40.2 MB/s) - ‘vocab’ saved [2996416/2996416]

--2021-01-23 21:47:32--  https://raw.githubusercontent.com/salesforce/ctrl/master/codes
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3332497 (3.2M) [text/plain]
Saving to: ‘codes’

codes               100%[===================>]   3.18M  --.-KB/s    in 0.1s    

2021-01-23 21:47:32 (32.5 MB/s) - ‘codes’ saved [3332497/3332497]

--2021-01-23 21:47:33--  https://raw.githubusercontent.com/salesfo

In [13]:
!python make_tf_records.py --text_file xy_train_validate.txt --control_code captions --sequence_len 256

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

### Download original model

In [14]:
!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt .

A newer version of gsutil (4.58) is available than the version you are
running (4.57). A detailed log of gsutil release changes is available
at https://pub.storage.googleapis.com/gsutil_ReleaseNotes.txt if you
would like to read them before updating.
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.index...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.meta...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/checkpoint...
| [4/4 files][ 12.7 GiB/ 12.7 GiB] 100% Done  41.3 MiB/s ETA 00:00:00           
Operation completed over 4 objects/12.7 GiB.                                     


### Patch to keras.py

In [15]:
!wget -O estimator.patch https://raw.githubusercontent.com/salesforce/ctrl/master/estimator.patch
!patch -b /anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch

--2021-01-23 21:51:13--  https://raw.githubusercontent.com/salesforce/ctrl/master/estimator.patch
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 820 [text/plain]
Saving to: ‘estimator.patch’

estimator.patch     100%[===================>]     820  --.-KB/s    in 0s      

2021-01-23 21:51:14 (66.8 MB/s) - ‘estimator.patch’ saved [820/820]

patching file /anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/keras.py
Hunk #1 succeeded at 50 (offset 3 lines).
Hunk #2 FAILED at 228.
Hunk #3 FAILED at 239.
Hunk #4 FAILED at 448.
Hunk #5 FAILED at 462.
4 out of 5 hunks FAILED -- saving rejects to file /anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/keras.py.rej


### Training modello originale con aggiuta del control_code

In [16]:
!python training.py --model_dir seqlen256_v1.ckpt/ --iterations 10

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tied_embedding_softmax (TiedEmb multiple             315810054   input_1[0][0]                    
                                                                 encoder[0][0]                    
__________________________

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
2021-01-23 21:53:03.967377: I tensorflow/core/common_runtime/direct_session.cc:296] Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from seqlen256_v1.ckpt/model.ckpt-413000
global_step/IsInitialized/VarIsInitializedOp: (VarIsInitializedOp): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.615573: I tensorflow/core/common_runtime/placer.cc:54] global_step/IsInitialized/VarIsInitializedOp: (V

2021-01-23 21:53:08.658406: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_7/layer_normalization_14/gamma/Initializer/ones: (Fill)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_7/layer_normalization_14/gamma/IsInitialized/VarIsInitializedOp: (VarIsInitializedOp): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.658418: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_7/layer_normalization_14/gamma/IsInitialized/VarIsInitializedOp: (VarIsInitializedOp)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_7/layer_normalization_14/gamma/Assign: (AssignVariableOp): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.658428: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_7/layer_normalization_14/gamma/Assign: (AssignVariableOp)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_7/layer_normalization_14/gamma/Read/ReadVariableOp: (ReadVariableOp): /job:

2021-01-23 21:53:08.712545: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_15/layer_normalization_30/batchnorm/mul_2: (Mul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_15/layer_normalization_30/batchnorm/ReadVariableOp: (ReadVariableOp): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.712560: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_15/layer_normalization_30/batchnorm/ReadVariableOp: (ReadVariableOp)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_15/layer_normalization_30/batchnorm/sub: (Sub): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.712576: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_15/layer_normalization_30/batchnorm/sub: (Sub)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_15/layer_normalization_30/batchnorm/add_1: (Add): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.712591: I tensorflow/co

2021-01-23 21:53:08.766856: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_23/multi_head_attention_23/Cast: (Cast)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_23/multi_head_attention_23/Sqrt: (Sqrt): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.766872: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_23/multi_head_attention_23/Sqrt: (Sqrt)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_23/multi_head_attention_23/truediv: (RealDiv): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.766887: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_23/multi_head_attention_23/truediv: (RealDiv)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_23/multi_head_attention_23/mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.766903: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_23/multi_head_attention_23

2021-01-23 21:53:08.821938: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_30/multi_head_attention_30/dense_180/BiasAdd: (BiasAdd)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_30/multi_head_attention_30/dense_181/kernel/Initializer/random_uniform/RandomUniform: (RandomUniform): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.821948: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_30/multi_head_attention_30/dense_181/kernel/Initializer/random_uniform/RandomUniform: (RandomUniform)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_30/multi_head_attention_30/dense_181/kernel/Initializer/random_uniform/sub: (Sub): /job:localhost/replica:0/task:0/device:CPU:0
2021-01-23 21:53:08.821956: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_30/multi_head_attention_30/dense_181/kernel/Initializer/random_uniform/sub: (Sub)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_l

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2021-01-23 21:53:19.565533: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 413000 into seqlen256_v1.ckpt/model.ckpt.
INFO:tensorflow:global_step/sec: 0.0171938
INFO:tensorflow:examples/sec: 0.0687752
INFO:tensorflow:global_step/sec: 0.0270655
INFO:tensorflow:examples/sec: 0.108262
INFO:tensorflow:global_step/sec: 0.0270322
INFO:tensorflow:examples/sec: 0.108129
INFO:tensorflow:global_step/sec: 0.0270587
INFO:

## Modificare generation.py e mettere nella riga 170: prompt = "Moby I" oppure prompt="captions the cat is on"
## Aggiungere nel control_code.py il code "Moby": 4256, oppure "captions": 1852

In [18]:
!python generation.py --model_dir seqlen256_v1.ckpt/

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tied_embedding_softmax (TiedEmb multiple             315810054   input_1[0][0]                    
                                                                 encoder[0][0]                    
__________________________

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2021-01-23 22:35:57.411494: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
Instructions for updating:
Use standard file utilities to get mtimes.
Loading vocabulary from vocab ...
Read 6086453827 words (246531 unique) from v

# Parte non testata/funzionante
### L'obiettivo è traferire da TensorFlow a PyTorch

In [75]:
!python -u convert_tf_to_huggingface_pytorch.py --tf ./seqlen256_v1.ckpt/model.ckpt-413010.data-00000-of-00001 --pytorch pytorch_checkpoint

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

In [78]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name . --temperature 0.5 --repetition 1.2 --prompt Legal

--2020-12-19 11:29:58--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 172.217.9.16, 142.250.68.176, 216.58.193.144, ...
Connecting to storage.googleapis.com|172.217.9.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:29:58 (17.1 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:29:58--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.06s   

2020-12-19 11:29:59 (36.5 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:29:

In [87]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name ctrl --temperature 0.5 --repetition 1.2 --prompt Legal --prefix Trump

--2020-12-19 11:49:57--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 142.250.68.176, 172.217.9.16, 172.217.12.48, ...
Connecting to storage.googleapis.com|142.250.68.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:49:57 (16.5 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:49:58--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.04s   

2020-12-19 11:49:58 (49.2 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:49

In [81]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name seqlen256_v1.ckpt/model.ckpt-413010.index --temperature 0.5 --repetition 1.2 --prompt Legal

--2020-12-19 11:32:04--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 172.217.9.176, 172.217.1.240, 172.217.14.176, ...
Connecting to storage.googleapis.com|172.217.9.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:32:04 (14.4 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:32:04--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.1s    

2020-12-19 11:32:05 (21.5 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:32